<table class="tfo-notebook-buttons" align="left">
  <td>
    <a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/deeplearning.ai/nlp/c3_w4_03_evaluate_siamese_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>    
  </td>
  <td>
    <a href="https://github.com/martin-fabbri/colab-notebooks/blob/master/deeplearning.ai/nlp/c3_w4_03_evaluate_siamese_model.ipynb" target="_parent"><img src="https://raw.githubusercontent.com/martin-fabbri/colab-notebooks/master/assets/github.svg" alt="View On Github"/></a>  </td>
</table>

# Evaluate a Siamese model: Ungraded Lecture Notebook


In [1]:
%%capture
!pip install trax

In [3]:
import trax.fastmath.numpy as np
!pip list | grep trax

trax                          1.3.7                


## Inspecting the necessary elements

In this lecture notebook you will learn how to evaluate a Siamese model using the accuracy metric. Because there are many steps before evaluating a Siamese network (as you will see in this week's assignment) the necessary elements and variables are replicated here using real data from the assignment:

   - `q1`: vector with dimension `(batch_size X max_length)` containing first questions to compare in the test set.
   - `q2`: vector with dimension `(batch_size X max_length)` containing second questions to compare in the test set.
   
   **Notice that for each pair of vectors within a batch $([q1_1, q1_2, q1_3, ...]$, $[q2_1, q2_2,q2_3, ...])$  $q1_i$ is associated to $q2_k$.**
        
        
   - `y_test`: 1 if  $q1_i$ and $q2_k$ are duplicates, 0 otherwise.
   
   - `v1`: output vector from the model's prediction associated with the first questions.
   - `v2`: output vector from the model's prediction associated with the second questions.

You can inspect each one of these variables by running the following cells:
